In [2]:
import io
import pickle

import psycopg2
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from PIL import Image
from sklearn.metrics import accuracy_score, classification_report
from torch.utils.data import Dataset
from tqdm.auto import tqdm

/home/roman/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/roman/.local/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
torch.cuda.empty_cache()
batch_size = 32
num_classes = 23
epochs = 15
#device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
transformation = transforms.Compose(
    [
        transforms.Resize((256, 256)),
        transforms.RandomRotation(20),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
    ]
)

base_transform = torchvision.transforms.Compose(
    [
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
    ]
)


class CustomDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __getitem__(self, index):
        img = self.data[index]
        if self.transform:
            img = self.transform(img)
        label = self.labels[index]
        return img, label

    def __len__(self):
        return len(self.data)


conn_select = psycopg2.connect(
    database="hse_medical",
    user='hse_medical',
    password='123456',
    host='127.0.0.1',
    port='5450',
    options="-c search_path=analyze_medical"
)

conn_select.autocommit = True

transform = transforms.ToTensor()


def get_connection():
    return conn_select


cursor = conn_select.cursor()

sql1 = f'''select 
    target,
    image from medical_pictures_train;'''
cursor.execute(sql1)
data_postgres = cursor.fetchall()
print(f"datatrain_size : {len(data_postgres)}")
cursor.close()
targets = []
images = []
for data in data_postgres:
    targets.append(data[0])
    image = Image.open(io.BytesIO(data[1]))
    images.append(image)

data_train = CustomDataset(images, targets, transform=transformation)
cursor_test = conn_select.cursor()

sql2 = f'''select 
    target,
    image from medical_pictures_test;'''
cursor_test.execute(sql2)
data_postgres_test = cursor_test.fetchall()
cursor_test.close()
targets_test = []
images_test = []

for data in data_postgres_test:
    targets_test.append(data[0])
    bytes_io = io.BytesIO(data[1])
    image_open = Image.open(bytes_io)
    images_test.append(image_open)

data_test = CustomDataset(images_test, targets_test, transform=base_transform)
train_dataloader = torch.utils.data.DataLoader(
    data_train, batch_size=batch_size, shuffle=True, num_workers=4
)
val_dataloader = torch.utils.data.DataLoader(
    data_test, batch_size=batch_size, shuffle=False, num_workers=4
)

model = torchvision.models.googlenet(pretrained=True)
model.to(device)

num_features = model.fc.in_features
model.fc = nn.Linear(num_features, num_classes)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.00001)


datatrain_size : 15588


/home/roman/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/roman/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:


for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(tqdm(train_dataloader), 0):
        inputs, labels = data
        optimizer.zero_grad()
        print(inputs,"inputs")
        print(inputs.shape,"inputs.shape")
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

    model.eval()
    train_preds = []
    train_labels = []
    with torch.no_grad():
        for images, labels in tqdm(train_dataloader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            train_preds.extend(predicted.cpu().numpy())
            train_labels.extend(labels.cpu().numpy())
    train_accuracy = accuracy_score(train_labels, train_preds)

    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {running_loss:.4f}, "
          f"Train Accuracy: {train_accuracy:.4f}")

print('Finished Training')
with open('googleNet.pickle', 'wb') as f:
    pickle.dump(model, f)
from sklearn.metrics import roc_auc_score, precision_score

correct = 0
total = 0
y_true = []
y_pred = []
predictions = []
with torch.no_grad():
    for data in tqdm(val_dataloader):
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(probabilities.cpu().numpy())

print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))
test_roc_auc = roc_auc_score(y_true, y_pred, multi_class='ovr')
test_classification_report = classification_report(y_true, predictions)
precision = precision_score(y_true, predictions, average='macro')
test_accuracy = accuracy_score(y_true, predictions)

print(f"Test Accuracy: {test_accuracy:.4f}, test precision {precision:.4f}"  f"Test ROC-AUC: {test_roc_auc:.4f}")

print(f"Classification Report:\n{test_classification_report}")

  0%|          | 0/488 [00:00<?, ?it/s]

tensor([[[[-1.9895, -1.9895, -1.9895,  ..., -1.9895, -1.9895, -1.9895],
          [-1.9895, -1.9895, -1.9895,  ..., -1.9895, -1.9895, -1.9895],
          [-1.9895, -1.9895, -1.9895,  ..., -1.9895, -1.9895, -1.9895],
          ...,
          [-1.9895, -1.9895, -1.9895,  ..., -1.9895, -1.9895, -1.9895],
          [-1.9895, -1.9895, -1.9895,  ..., -1.9895, -1.9895, -1.9895],
          [-1.9895, -1.9895, -1.9895,  ..., -1.9895, -1.9895, -1.9895]],

         [[-1.9844, -1.9844, -1.9844,  ..., -1.9844, -1.9844, -1.9844],
          [-1.9844, -1.9844, -1.9844,  ..., -1.9844, -1.9844, -1.9844],
          [-1.9844, -1.9844, -1.9844,  ..., -1.9844, -1.9844, -1.9844],
          ...,
          [-1.9844, -1.9844, -1.9844,  ..., -1.9844, -1.9844, -1.9844],
          [-1.9844, -1.9844, -1.9844,  ..., -1.9844, -1.9844, -1.9844],
          [-1.9844, -1.9844, -1.9844,  ..., -1.9844, -1.9844, -1.9844]],

         [[-1.7107, -1.7107, -1.7107,  ..., -1.7107, -1.7107, -1.7107],
          [-1.7107, -1.7107, -